# Análisis de sentimientos con reviews de productos de Amazon España (opcional)

Si has hecho ya el ejercicio de web scraping con `Requests` y `BeautifulSoup` habrás visto cómo extraer datos de una página web.

El dataset que utilizarás en este ejercicio (que no es obligatorio entregar) lo he generado utilizando `Scrapy` y `BeautifulSoup`, y contiene unas $700.000$ entradas con dos columnas: el número de estrellas dadas por un usuario a un determinado producto y el comentario sobre dicho producto; exactamente igual que en el ejercico de scraping.

Ahora, tu objetivo es utilizar técnicas de procesamiento de lenguaje natural para hacer un clasificador que sea capaz de distinguir (¡y predecir!) si un comentario es positivo o negativo.

Es un ejercicio MUY complicado, más que nada porque versa sobre técnicas que no hemos visto en clase. Así que si quieres resolverlo, te va a tocar estudiar y *buscar por tu cuenta*; exactamente igual que como sería en un puesto de trabajo. Dicho esto, daré un par de pistas:

+ El número de estrellas que un usuario da a un producto es el indicador de si a dicho usuario le ha gustado el producto o no. Una persona que da 5 estrellas (el máximo) a un producto probablemente esté contento con él, y el comentario será por tanto positivo; mientras que cuando una persona da 1 estrella a un producto es porque no está satisfecha... 
+ Teniendo el número de estrellas podríamos resolver el problema como si fuera de regresión; pero vamos a establecer una regla para convertirlo en problema de clasificación: *si una review tiene 4 o más estrellas, se trata de una review positiva; y será negativa si tiene menos de 4 estrellas*. Así que probablemente te toque transformar el número de estrellas en otra variable que sea *comentario positivo/negativo*.

Y... poco más. Lo más complicado será convertir el texto de cada review en algo que un clasificador pueda utilizar y entender (puesto que los modelos no entienden de palabras, sino de números). Aquí es donde te toca investigar las técnicas para hacerlo. El ejercicio se puede conseguir hacer, y obtener buenos resultados, utilizando únicamente Numpy, pandas y Scikit-Learn; pero siéntete libre de utilizar las bibliotecas que quieras.

Ahora escribiré una serie de *keywords* que probablemente te ayuden a saber qué buscar:

`bag of words, tokenizer, tf, idf, tf-idf, sklearn.feature_extraction, scipy.sparse, NLTK (opcional), stemmer, lemmatizer, stop-words removal, bigrams, trigrams`

No te desesperes si te encuentras muy perdido/a y no consigues sacar nada. Tras la fecha de entrega os daré un ejemplo de solución explicado con todo el detalle posible.

¡Ánimo y buena suerte!

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import math
import string
from __future__ import division

In [30]:
amazon_rev = pd.read_csv('amazon_es_reviews.csv', sep=';')

In [31]:
amazon_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702446 entries, 0 to 702445
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   comentario  702446 non-null  object 
 1   estrellas   702446 non-null  float64
dtypes: float64(1), object(1)
memory usage: 10.7+ MB


In [32]:
amazon_rev['tipo_rev'] = np.where(amazon_rev['estrellas']>=4 , 1, -1)

In [33]:
amazon_rev

,comentario,estrellas,tipo_rev
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,-1
2,"Razonablemente bien escrito, bien ambientado, ...",3.0,-1
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1
4,A simple vista m parecia una buena camara pero...,1.0,-1
...,...,...,...
702441,Nada a destacar. Muy bien. Envio correcto rapi...,5.0,1
702442,"A falta de probarlo, presenta signos de haber ...",3.0,-1
702443,Si es un simple cargador muy bien de precio y ...,3.0,-1
702444,"Si el producto no está mal , lo único que no e...",4.0,1


In [34]:
def quitar_simbolos(text):
    return text.translate(str.maketrans('','',string.punctuation))

In [37]:
amazon_rev['comentarios_no_simbolos'] = amazon_rev['comentario'].apply(quitar_simbolos)

In [38]:
amazon_rev

,comentario,estrellas,tipo_rev,comentarios_no_simbolos
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1,Para chicas es perfecto ya que la esfera no es...
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,-1,Muy floja la cuerda y el anclaje es de mala ca...
2,"Razonablemente bien escrito, bien ambientado, ...",3.0,-1,Razonablemente bien escrito bien ambientado qu...
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1,Hola No suel o escribir muchas opiniones sobre...
4,A simple vista m parecia una buena camara pero...,1.0,-1,A simple vista m parecia una buena camara pero...
...,...,...,...,...
702441,Nada a destacar. Muy bien. Envio correcto rapi...,5.0,1,Nada a destacar Muy bien Envio correcto rapido...
702442,"A falta de probarlo, presenta signos de haber ...",3.0,-1,A falta de probarlo presenta signos de haber s...
702443,Si es un simple cargador muy bien de precio y ...,3.0,-1,Si es un simple cargador muy bien de precio y ...
702444,"Si el producto no está mal , lo único que no e...",4.0,1,Si el producto no está mal lo único que no es...


In [43]:
amazon_rev_man = amazon_rev[amazon_rev['estrellas'] != 3]

In [44]:
amazon_rev_man

,comentario,estrellas,tipo_rev,comentarios_no_simbolos
0,"Para chicas es perfecto, ya que la esfera no e...",4.0,1,Para chicas es perfecto ya que la esfera no es...
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0,-1,Muy floja la cuerda y el anclaje es de mala ca...
3,Hola! No suel o escribir muchas opiniones sobr...,5.0,1,Hola No suel o escribir muchas opiniones sobre...
4,A simple vista m parecia una buena camara pero...,1.0,-1,A simple vista m parecia una buena camara pero...
5,"NI para pasar el rato, los personajes no tiene...",1.0,-1,NI para pasar el rato los personajes no tienen...
...,...,...,...,...
702439,El paquete es muy rápido llegó a mi casa solo ...,5.0,1,El paquete es muy rápido llegó a mi casa solo ...
702440,"Le ha encantado a mi niña de 4 años, tiene un ...",4.0,1,Le ha encantado a mi niña de 4 años tiene un m...
702441,Nada a destacar. Muy bien. Envio correcto rapi...,5.0,1,Nada a destacar Muy bien Envio correcto rapido...
702444,"Si el producto no está mal , lo único que no e...",4.0,1,Si el producto no está mal lo único que no es...


In [45]:
amazon_rev_man = amazon_rev_man.drop("comentario", axis=1)

In [46]:
amazon_rev_man

,estrellas,tipo_rev,comentarios_no_simbolos
0,4.0,1,Para chicas es perfecto ya que la esfera no es...
1,1.0,-1,Muy floja la cuerda y el anclaje es de mala ca...
3,5.0,1,Hola No suel o escribir muchas opiniones sobre...
4,1.0,-1,A simple vista m parecia una buena camara pero...
5,1.0,-1,NI para pasar el rato los personajes no tienen...
...,...,...,...
702439,5.0,1,El paquete es muy rápido llegó a mi casa solo ...
702440,4.0,1,Le ha encantado a mi niña de 4 años tiene un m...
702441,5.0,1,Nada a destacar Muy bien Envio correcto rapido...
702444,4.0,1,Si el producto no está mal lo único que no es...


In [52]:
train,test=train_test_split(amazon_rev_man,train_size=0.8,random_state=42)

In [ ]:
col_indep=[x for x in test.columns if x !="tipo_rev"]
target="tipo_rev"

X_train=train[["contador_pal","estrellas"]]
y_train=train[target]
X_test=test[col_indep]
y_test=test[target]